In [18]:
# Basics
import numpy as np
import pandas as pd
import pickle
import h5py

In [19]:
name = "debug_cnn_cat"
classification = False
timesteps = 1
batch_size = 20
debug = True

In [20]:
with open("./data/partition.pkl", 'rb') as f:
    partition = pickle.load(f)
    
if debug:
    partition["train"] = partition["train"][:20]
    partition["valid"] = partition["valid"][:5]

# Load labels

In [21]:
target = pd.read_csv("./data/ERU_Scores_Ids_5-Scans_Validity-0_VisuallyScored.csv")
labels = target.set_index("StId").to_dict()["ERU.M2"]

In [22]:
if classification:
    # combine 0+1 as 0 = no emph in scan, 1 = no emph in region
    label_converter = {0: 0, 1: 0, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
else:
    # Rescale to [0;1]; merge category 0+1
    label_converter = {0: 0.0, 1: 0.0, 2: 0.03, 3: 0.155, 4: 0.38, 5: 0.63, 6: 0.88}
    # label_converter = {0: 0, 1: 0, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
labels = {key: label_converter[val] for key, val in labels.items()}

# Load data

In [23]:
from scripts.data_gen import DataGenerator

In [24]:
gen = DataGenerator(labels, partition, mode="train", oversample=True, 
                    classes=1, batch_size=5, timesteps=1)

In [25]:
from keras.utils import to_categorical

In [32]:
set(list(labels.values()))

{0.0, 0.03, 0.155, 0.38, 0.63, 0.88}

In [34]:
to_categorical(0, num_classes=6)

array([1., 0., 0., 0., 0., 0.])

In [7]:
# predGen = DataGenerator("train", classification=classification, batch_size=batch_size,
#                          timesteps=timesteps, channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=True)
# predGen = predGen.generate(labels, partition["train"])

# Load model

In [8]:
from keras.models import load_model
from glob import glob

Using TensorFlow backend.


In [9]:
path = sorted(glob("./output/"+name+"/weights/*.hdf5"))[-1]
print(path)

model = load_model(path)

./output/debug_cnn_cat/weights/epoch_030_final.hdf5


In [10]:
X, y = predGen.__next__()
y_ = model.predict_on_batch(X)

In [11]:
if classification:
    actual = np.argmax(y, axis=1)
    predicted = np.argmax(y_, axis=1)
    
    comp = np.column_stack((np.argmax(y, axis=1), np.argmax(y_, axis=1)))
else:
    comp = np.empty((batch_size,2))
    comp[:,0] = np.reshape(y*100, (batch_size))
    comp[:,1] = np.reshape(y_*100, (batch_size))

# left: actual, right: predicted
print(comp)

[[0 0]
 [2 2]
 [0 0]
 [0 0]
 [2 2]
 [2 2]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [2 2]
 [0 0]
 [3 3]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


In [12]:
y

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])